In [4]:
import numpy as np
import os
from stable_baselines3 import SAC
from ot2_env_wrapper import OT2Env
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize


def test_random_positions(model_path, vec_norm_path, num_positions=10):
    """
    Tests the arm's accuracy at reaching random positions within the workspace.
    """
    # Create environment
    env = DummyVecEnv([lambda: OT2Env(render=False)])
    
    # Load normalization stats
    if not os.path.exists(vec_norm_path):
        raise FileNotFoundError(f"Could not find normalization stats at {vec_norm_path}")
    
    env = VecNormalize.load(vec_norm_path, env)
    env.training = False
    env.norm_reward = False
    
    # Load model (SAC)
    model = SAC.load(model_path)
    
    print("\n" + "="*50)
    print(f"TESTING {num_positions} RANDOM POSITIONS")
    print("="*50)
    
    errors = []
    successes = 0
    
    # Set random seed for reproducibility
    np.random.seed(42)
    
    # Define actual work envelope with safety margin
    # Original limits from your JSON
    x_min, x_max = -0.187, 0.2532
    y_min, y_max = -0.1706, 0.2196
    z_min, z_max = 0.1694, 0.2895
    
    # Add 10mm (0.01m) safety margin on all sides
    margin = 0.05
    safe_x_min, safe_x_max = x_min + margin, x_max - margin
    safe_y_min, safe_y_max = y_min + margin, y_max - margin
    safe_z_min, safe_z_max = z_min + margin, z_max - margin
    
    print(f"\nWork Envelope (with {margin*1000:.0f}mm margin):")
    print(f"  X: [{safe_x_min:.3f}, {safe_x_max:.3f}]")
    print(f"  Y: [{safe_y_min:.3f}, {safe_y_max:.3f}]")
    print(f"  Z: [{safe_z_min:.3f}, {safe_z_max:.3f}]")
    print("-"*50)
    
    for i in range(num_positions):
        # Generate random goal position within safe work envelope
        random_goal = np.array([
            np.random.uniform(safe_x_min, safe_x_max),
            np.random.uniform(safe_y_min, safe_y_max),
            np.random.uniform(safe_z_min, safe_z_max)
        ]).astype(np.float32)
        
        print(f"\nTest {i+1}: Goal [{random_goal[0]:.3f}, {random_goal[1]:.3f}, {random_goal[2]:.3f}]")
        
        # Reset environment
        obs = env.reset()
        
        # Override the goal position
        env.envs[0].goal_position = random_goal
        
        # Update observation with new goal
        obs[0][3:6] = random_goal
        
        done = False
        steps = 0
        max_steps = 1000
        
        while not done and steps < max_steps:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, terminated, info = env.step(action)
            steps += 1
            
            if terminated[0] or steps >= max_steps:
                # Get final position
                if terminated[0]:
                    terminal_obs = info[0]['terminal_observation']
                    real_obs = env.unnormalize_obs(terminal_obs)
                else:
                    real_obs = env.unnormalize_obs(obs[0])
                
                final_pipette_pos = real_obs[:3]
                
                # Calculate error
                error = np.linalg.norm(final_pipette_pos - random_goal)
                errors.append(error)
                
                if error < 0.001:
                    successes += 1
                    print(f"  ✓ Success! Error: {error*1000:.3f}mm in {steps} steps")
                else:
                    print(f"  ✗ Error: {error*1000:.3f}mm in {steps} steps")
                
                done = True
    
    env.close()
    
    # Calculate statistics
    avg_error = np.mean(errors)
    std_error = np.std(errors)
    min_error = np.min(errors)
    max_error = np.max(errors)
    success_rate = (successes / num_positions) * 100
    
    # Print summary
    print("\n" + "="*50)
    print("SUMMARY")
    print("="*50)
    print(f"Success Rate:      {success_rate:.1f}% ({successes}/{num_positions})")
    print(f"Average Error:     {avg_error*1000:.2f}mm ± {std_error*1000:.2f}mm")
    print(f"Best Accuracy:     {min_error*1000:.3f}mm")
    print(f"Worst Accuracy:    {max_error*1000:.3f}mm")
    print("="*50)


if __name__ == "__main__":
    test_random_positions(
        model_path="C:/Users/honya/Documents/GitHub/RL-Group-1/Gergo/final_model.zip",
        vec_norm_path="C:/Users/honya/Documents/GitHub/RL-Group-1/Gergo/vec_normalize.pkl",
        num_positions=20
    )


TESTING 20 RANDOM POSITIONS

Work Envelope (with 50mm margin):
  X: [-0.137, 0.203]
  Y: [-0.121, 0.170]
  Z: [0.219, 0.239]
--------------------------------------------------

Test 1: Goal [-0.010, 0.155, 0.234]
  ✓ Success! Error: 0.593mm in 98 steps

Test 2: Goal [0.067, -0.075, 0.223]
  ✓ Success! Error: 0.792mm in 44 steps

Test 3: Goal [-0.117, 0.131, 0.231]
  ✓ Success! Error: 0.944mm in 51 steps

Test 4: Goal [0.104, -0.115, 0.239]
  ✓ Success! Error: 0.922mm in 78 steps

Test 5: Goal [0.146, -0.059, 0.223]
  ✓ Success! Error: 0.879mm in 53 steps

Test 6: Goal [-0.075, -0.032, 0.230]
  ✓ Success! Error: 0.740mm in 80 steps

Test 7: Goal [0.010, -0.036, 0.232]
  ✓ Success! Error: 0.824mm in 56 steps

Test 8: Goal [-0.090, -0.036, 0.227]
  ✓ Success! Error: 0.686mm in 100 steps

Test 9: Goal [0.018, 0.107, 0.223]
  ✓ Success! Error: 0.908mm in 160 steps

Test 10: Goal [0.038, 0.051, 0.220]
  ✓ Success! Error: 0.823mm in 28 steps

Test 11: Goal [0.070, -0.071, 0.221]
  ✓ Success!